In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# pip install python-docx pytesseract
# from docx import Document
from PIL import Image
import pytesseract
import os
import glob 
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset/data/train.csv
/kaggle/input/dataset/data/test.csv
/kaggle/input/dataset/data/test/24158401-Rental-Agreement.png
/kaggle/input/dataset/data/test/95980236-Rental-Agreement.png
/kaggle/input/dataset/data/test/228094620-Rental-Agreement.pdf.docx
/kaggle/input/dataset/data/test/156155545-Rental-Agreement-Kns-Home.pdf.docx
/kaggle/input/dataset/data/train/6683127-House-Rental-Contract-GERALDINE-GALINATO-v2-Page-1.docx
/kaggle/input/dataset/data/train/18325926-Rental-Agreement-1.docx
/kaggle/input/dataset/data/train/46239065-Standard-Rental-Agreement-Rental-With-Performance-Fee.docx
/kaggle/input/dataset/data/train/50070534-RENTAL-AGREEMENT.docx
/kaggle/input/dataset/data/train/6683129-House-Rental-Contract-Geraldine-Galinato-v2.docx
/kaggle/input/dataset/data/train/47854715-RENTAL-AGREEMENT.docx
/kaggle/input/dataset/data/train/54945838-Rental-Agreement.png
/kaggle/input/dataset/data/train/36199312-Rental-Agreement.png
/kaggle/input/dataset/data/train/44737744-Maddired

In [4]:
Train = "/kaggle/input/dataset/data/train"
Test = "/kaggle/input/dataset/data/test"
train_df = pd.read_csv("/kaggle/input/dataset/data/train.csv")
test_df = pd.read_csv("/kaggle/input/dataset/data/test.csv")

def find_real_file(base_dir, partial_name):
    matches = glob.glob(os.path.join(base_dir, partial_name + "*"))
    if matches:
        return matches[0]
    else:
        return None

train_df["full_path"] = train_df["File Name"].apply(lambda x: find_real_file(Train, x))
test_df["full_path"] = test_df["File Name"].apply(lambda x: find_real_file(Test, x))

train_df["file_type"] = train_df["full_path"].apply(lambda x: x.split(".")[-1] if pd.notna(x) else "missing")
test_df["file_type"] = test_df["full_path"].apply(lambda x: x.split(".")[-1] if pd.notna(x) else "missing")

train_png_df = train_df[train_df["file_type"] == "png"].copy()
test_png_df = test_df[test_df["file_type"] == "png"].copy()


print("File types in train")
print(train_df["file_type"].value_counts())

print("File types test")
print(test_df["file_type"].value_counts())

File types in train
file_type
docx       6
png        3
missing    1
Name: count, dtype: int64
File types test
file_type
png     2
docx    2
Name: count, dtype: int64


In [22]:
def extract_text_from_png(png_path):
    try:
        image = Image.open(png_path)
        text = pytesseract.image_to_string(image)
        return text.strip()
    except Exception as e:
        return f"Error reading {png_path}: {str(e)}"

train_png_df["extracted_text"] = train_png_df["full_path"].apply(extract_text_from_png)
test_png_df["extracted_text"] = test_png_df["full_path"].apply(extract_text_from_png)

print("Extracted text from png(train):")
for idx, row in train_png_df.iterrows():
    print(f" Document{idx}" )
    print(row["extracted_text"])

Extracted text from png(train):
 Document4
RENEWAL OF RENTAL AGREEMENT

This AGREEMENT of Rent is made in Bangalore and Executed today the Ist" of May
2010

BY AND BETWEEN

1. Mr. Balaji.R
Aged about 63 years,
No 24 2" Cross, SBM
Colony Mathikere - 560054

Hereinafter referred and called as the ‘Lessor’ of the First part of one part:
//AND//
1 Mr.Kartheek R
Aged about 31 years,
No.81, sri manjunatha nilaya, raju colony,
yamalur
B angalore-560037.

Hereinafter referred and called as the ‘Lessees’ of the second part of the another part:

NOW THIS AGREEMENT OF RENT WITNESSETH TN AS FOLLOWS:

Whereas the first party is the sole and absolute owner of the above cited / scheduled
premises is hereby continued to be rented out the same to the second party which terms
and conditions is as follows:

The lessor have received a security Deposit amount of Rs.40,000/- (Rupees Fourty
Thousand only) from the Lessees and hereby acknowledges the receipt of the same, which
carries no interest but to be re

In [9]:
test_df.head()

,File Name,Aggrement Value,Aggrement Start Date,Aggrement End Date,Renewal Notice (Days),Party One,Party Two,full_path,file_type
0,24158401-Rental-Agreement,12000,01.04.2008,31.03.2009,60,Hanumaiah,Vishal Bhardwaj,/kaggle/input/dataset/data/test/24158401-Renta...,png
1,95980236-Rental-Agreement,9000,01.04.2010,31.03.2011,30,S.Sakunthala,V.V.Ravi Kian,/kaggle/input/dataset/data/test/95980236-Renta...,png
2,156155545-Rental-Agreement-Kns-Home,12000,15.12.2012,14.11.2013,30,V.K.NATARAJ,VYSHNAVI DAIRY SPECIALITIES Private Ltd,/kaggle/input/dataset/data/test/156155545-Rent...,docx
3,228094620-Rental-Agreement,15000,07.07.2013,06.06.2014,30,KAPIL MEHROTRA,.B.Kishore,/kaggle/input/dataset/data/test/228094620-Rent...,docx


In [10]:
print(test_df.columns)

Index(['File Name', 'Aggrement Value', 'Aggrement Start Date',
       'Aggrement End Date', 'Renewal Notice (Days)', 'Party One', 'Party Two',
       'full_path', 'file_type'],
      dtype='object')


In [19]:
fields = [
    "Aggrement Value",
    "Aggrement Start Date",
    "Aggrement End Date",
    "Renewal Notice (Days)",
    "Party One",
    "Party Two"
]
recall_scores = {}

for field in fields:
    true_posi = 0
    false_nega = 0

    for idx, row in test_png_df.iterrows():
        true_val = str(row.get(field, "")).strip().lower()
        ext_text = row["extracted_text"].lower()

        if true_val == "":
            continue

        if true_val in ext_text:
            true_posi += 1
        else:
            false_nega += 1

    total = true_posi + false_nega
    recall = true_posi / total if total > 0 else 0
    recall_scores[field] = recall

In [21]:
summary= pd.DataFrame.from_dict(recall_scores, orient='index', columns=['recall'])
summary.index.name = 'Field Name'
summary =summary.reset_index()
print(summary)

              Field Name  recall
0        Aggrement Value     0.5
1   Aggrement Start Date     0.0
2     Aggrement End Date     0.0
3  Renewal Notice (Days)     1.0
4              Party One     1.0
5              Party Two     0.5
